In [4]:
"""
Created on Tues Nov 25

@author: Simon H
"""

import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score
import time

In [5]:
train_start = time.time()  

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [6]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 128)
        self.output = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.output(x)
        return x

In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_set = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)

test_set = datasets.MNIST(root='./data', train=False, transform=transform)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

model = SimpleNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

100%|██████████| 9.91M/9.91M [00:01<00:00, 6.03MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 159kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.51MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.74MB/s]


In [8]:
def compute_metrics(y_true, y_pred):
    y_true = y_true.cpu().numpy()
    y_pred = y_pred.cpu().numpy()

    acc = (y_true == y_pred).mean()
    precision = precision_score(y_true, y_pred, average="macro")
    recall = recall_score(y_true, y_pred, average="macro")
    f1 = f1_score(y_true, y_pred, average="macro")

    return acc, precision, recall, f1

In [9]:
for epoch in range(5):
    model.train()
    epoch_labels = []
    epoch_preds = []
    running_loss = 0

    for images, labels in train_loader:
        images, labels = images.view(-1, 28*28).to(device), labels.to(device)

        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = torch.max(output, 1)
        epoch_labels.append(labels)
        epoch_preds.append(predicted)

    epoch_labels = torch.cat(epoch_labels)
    epoch_preds = torch.cat(epoch_preds)

    acc, precision, recall, f1 = compute_metrics(epoch_labels, epoch_preds)

    print(f"Epoch {epoch+1}/5 | "
          f"Loss: {running_loss/len(train_loader):.4f} | "
          f"Acc: {acc:.4f} | Prec: {precision:.4f} | Recall: {recall:.4f} | F1: {f1:.4f}")

Epoch 1/5 | Loss: 0.3745 | Acc: 0.8871 | Prec: 0.8863 | Recall: 0.8855 | F1: 0.8857
Epoch 2/5 | Loss: 0.1769 | Acc: 0.9457 | Prec: 0.9452 | Recall: 0.9452 | F1: 0.9452
Epoch 3/5 | Loss: 0.1303 | Acc: 0.9593 | Prec: 0.9590 | Recall: 0.9590 | F1: 0.9590
Epoch 4/5 | Loss: 0.1018 | Acc: 0.9682 | Prec: 0.9679 | Recall: 0.9679 | F1: 0.9679
Epoch 5/5 | Loss: 0.0855 | Acc: 0.9728 | Prec: 0.9727 | Recall: 0.9726 | F1: 0.9727


In [10]:
def evaluate_model(model, data_loader):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.view(-1, 28*28).to(device), labels.to(device)
            output = model(images)
            _, predicted = torch.max(output, 1)

            all_labels.append(labels)
            all_preds.append(predicted)

    all_labels = torch.cat(all_labels).cpu()
    all_preds = torch.cat(all_preds).cpu()

    accuracy = (all_labels == all_preds).float().mean().item()

    precision = precision_score(all_labels, all_preds, average='macro')
    recall    = recall_score(all_labels, all_preds, average='macro')
    f1        = f1_score(all_labels, all_preds, average='macro')

    print(f"Accuracy : {accuracy*100:.2f}%")
    print(f"Precision: {precision:.4f}")
    print(f"Recall   : {recall:.4f}")
    print(f"F1-score : {f1:.4f}")

In [11]:
print("\nModel Evaluation on Test Set")
evaluate_model(model, test_loader)


Model Evaluation on Test Set
Accuracy : 96.72%
Precision: 0.9671
Recall   : 0.9672
F1-score : 0.9669
